In [2]:
import numpy as np
import os
import sys
import warnings

warnings.filterwarnings("ignore")

Endian = '<' #Litte Endian:<, Big Endian:>
###############################################################################
# Sort filename_list based on iteration
###############################################################################
Input_Args  = sys.argv
Input_Directory = '/scratch/j/jphickey/jwawryk/M0.3' #= Input_Args[1] # Directory of .res files
Output_Directory = '/home/j/jphickey/jwawryk/ME765_Project' #Input_Args[2] # Directory of output files
File_Step = 1 #int(Input_Args[3]) # Skipping number of files. e.g. 5 means [0, 5, 10 ,... 200] 
#File_Step = 1
os.chdir(Input_Directory)
dot_file_extension = ".res"
listall = list(set(os.listdir(Input_Directory)))		
filename_list = []					

for filename in listall:
    if filename[-(len(dot_file_extension)):] == dot_file_extension:		
        filename_list.append(filename)
        
Iteration = np.zeros((len(filename_list)))
for Counter, filename in enumerate(filename_list):
    Iteration[Counter] = int(filename[5:12])
        
Sorted_indices = np.argsort(Iteration)
Temp = np.empty_like(filename_list)
for i in range(len(Sorted_indices)):
    Temp[i] = filename_list[Sorted_indices[i]]
filename_list = Temp[::File_Step] # Python sequence slice addresses can be written as a[start:end:step]

Iteration_From = 0 #int(Input_Args[4]) # File limit, From.
Iteration_To = 4800 #int(Input_Args[5]) # File limit, To
###############################################################################
# Initial Setup
###############################################################################
w30 = 9.487209
#w30 = float(Input_Args[6])
#w30 = 9.487209
Circ0 = 1.159780
#Circ0 = float(Input_Args[7])
#Circ0 = 1.159780
M0 = 0.3
#M0 = float(Input_Args[8])
#M0 = 0.5
Mu0 = 0.0007732
#Mu0 = float(Input_Args[9])
#Mu0 = 0.0007732
Pr = 0.7
#Pr = float(Input_Args[10])
#Pr = 0.7
gamma = 1.4
gmlinv = 1.0 / (gamma - 1.0)
rho0 = 1.
U0 = 1.
c0 = U0 / M0
L0 = 1.
R0 = 1./(gamma*M0*M0)
P0 = 1./(gamma*M0*M0)
T0 = P0 / (rho0*R0)
Dil0 = U0 / L0
Lamb0 = w30*U0
Helicity0 = U0*w30
###############################################################################
# Geometry (Grid) setup
###############################################################################
os.chdir(Input_Directory)
with open(filename_list[0],'rb') as fin:
    Index0 = 0
    Index0 = np.dtype('int64').type(Index0)
    fin.seek(Index0)
    Int_Header = np.fromfile(fin, Endian+'i', 12)
    Preci = Int_Header[0]
    Preci = np.dtype('int64').type(Preci)
    if Preci == 4:
        DType = 'float32' # I assume float32 is Preci (4) bytes
    else:
        DType = 'float64'
    Nx = Int_Header[3] # Nx Original
    Ny = Int_Header[4]
    Nz = Int_Header[5]
    Nx = np.dtype('int64').type(Nx)
    Ny = np.dtype('int64').type(Ny)
    Nz = np.dtype('int64').type(Nz)
               
    Index0 = 12*4 #12 integer numbers
    fin.seek(Index0)
    Double_Hearder = np.fromfile(fin, Endian+'d', 1+Nx+Ny+Nz)#'double'
    x = Double_Hearder[1:Nx+1]
    y = Double_Hearder[Nx+1:Nx+Ny+1]
    z = Double_Hearder[Nx+Ny+1:Nx+Ny+Nz+1]
    
    XOS = -1.2 # X-position Of Surface (Down)
    X0_Index = (np.abs(x - XOS)).argmin()
    if XOS < x[X0_Index]:
        X0_Index = X0_Index - 1
    
    XOS = 5.2 # X-position Of Surface (Up)
    X1_Index = (np.abs(x - XOS)).argmin()
    if XOS < x[X1_Index]:
        X1_Index = X1_Index - 1
    
    X0_Index = np.dtype('int64').type(X0_Index)
    X1_Index = np.dtype('int64').type(X1_Index)
    
    Index0_Main = 0
    Index0_Main = np.dtype('int64').type(Index0_Main)
    
    X0_Index_Original = X0_Index
    X1_Index_Original = X1_Index
    
    Y0_Index = int(Ny/2)-int(Nz/2)
    Y1_Index = int(Ny/2)+int(Nz/2)
    
    NOL = X1_Index - X0_Index
###############################################################################
# Main Analysis
###############################################################################
Time = np.zeros((len(filename_list)))
for Counter, filename in enumerate(filename_list):
    if Iteration_From <= int(filename[5:12]) < Iteration_To:
        os.chdir(Input_Directory)
        print('Post-processing of ',filename)
        with open(filename,'rb') as fin:
            ###############################################################################
            # Restore geometry
            ###############################################################################
            y_Block, x_Block, z_Block = np.meshgrid(y[Y0_Index:Y1_Index], x[X0_Index:X1_Index], z)
            ###############################################################################
            # Time
            ###############################################################################
            Index0 = 12*4 #12 integer numbers
            fin.seek(Index0)
            Double_Hearder = np.fromfile(fin, Endian+'d', 1) # 'd' means 'double'
            Time[Counter] = Double_Hearder[0]
            ###############################################################################
            # Extracting primitive variables from the .res file on specific blocks
            ###############################################################################
            Index0 = 12*4 #12 integer numbers
            
            # rho
            Index0 = Index0 + (Nx+Ny+Nz+1)*8 #one double (for time) + Nx for x, Ny for y, Nz for z
            Index0_Main = Index0 + ((X0_Index)*Ny*Nz)*Preci
            fin.seek(Index0_Main)
            rho = np.reshape(np.fromfile(fin, DType, NOL*Ny*Nz), (NOL,Ny,Nz))[:,Y0_Index:Y1_Index,:]
    
            # u
            Index0 = Index0 + (Nx*Ny*Nz)*Preci
            Index0_Main = Index0 + ((X0_Index)*Ny*Nz)*Preci
            fin.seek(Index0_Main)
            u = np.reshape(np.fromfile(fin, DType, NOL*Ny*Nz), (NOL,Ny,Nz))[:,Y0_Index:Y1_Index,:] / rho
            
            # v
            Index0 = Index0 + (Nx*Ny*Nz)*Preci
            Index0_Main = Index0 + ((X0_Index)*Ny*Nz)*Preci
            fin.seek(Index0_Main)
            v = np.reshape(np.fromfile(fin, DType, NOL*Ny*Nz), (NOL,Ny,Nz))[:,Y0_Index:Y1_Index,:] / rho
            
            # w
            Index0 = Index0 + (Nx*Ny*Nz)*Preci
            Index0_Main = Index0 + ((X0_Index)*Ny*Nz)*Preci
            fin.seek(Index0_Main)
            w = np.reshape(np.fromfile(fin, DType, NOL*Ny*Nz), (NOL,Ny,Nz))[:,Y0_Index:Y1_Index,:] / rho
            
            # P
            Index0 = Index0 + (Nx*Ny*Nz)*Preci
            Index0_Main = Index0 + ((X0_Index)*Ny*Nz)*Preci
            fin.seek(Index0_Main)
            P = (np.reshape(np.fromfile(fin, DType, NOL*Ny*Nz), (NOL,Ny,Nz))[:,Y0_Index:Y1_Index,:] - 0.5*rho*(u**2+v**2+w**2)) / gmlinv
            
print("DONE")

Post-processing of  end_n0000040_t1.000e-01.res
Post-processing of  end_n0000080_t2.000e-01.res
Post-processing of  end_n0000120_t3.000e-01.res
Post-processing of  end_n0000160_t4.000e-01.res
Post-processing of  end_n0000200_t5.000e-01.res
Post-processing of  end_n0000240_t5.999e-01.res
Post-processing of  end_n0000280_t6.999e-01.res


KeyboardInterrupt: 